<div style="text-align:center; font-size:36px; font-weight:bold; color:#4A4A4A; background-color:#fff6e4; padding:10px; border:3px solid #f5ecda; border-radius:6px">
    Medical Cost Prediction
    <p style="text-align:center; font-size:14px; font-weight:normal; color:#4A4A4A; margin-top:12px;">
        Author: Jens Bender <br> 
        Created: December 2025<br>
        Last updated: January 2026
    </p>
</div>

<div style="background-color:#2c699d; color:white; padding:15px; border-radius:6px;">
    <h1 style="margin:0px">Imports</h1>
</div>

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data preprocessing (Scikit-learn)
from sklearn.preprocessing import (
    StandardScaler, 
    OneHotEncoder, 
    OrdinalEncoder
)

# Model selection
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import randint, uniform  # for random hyperparameter values

# Models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor 

# Model evaluation
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_percentage_error, 
    r2_score
)

<div style="background-color:#2c699d; color:white; padding:15px; border-radius:6px;">
    <h1 style="margin:0px">Data Loading and Inspection</h1>
</div>
<div style="background-color:#fff6e4; padding:15px; border:3px solid #f5ecda; border-radius:6px;">
    📌 Load the MEPS-HC 2023 data from the <code>h251.sas7bdat</code> file (SAS V9 format) into a Pandas DataFrame.
</div>

In [2]:
try:
    # Load data using 'latin1' encoding because MEPS SAS files don't store text as UTF-8 and instead use Western European (ISO-8859-1), also known as latin1.
    df = pd.read_sas("../data/h251.sas7bdat", format="sas7bdat", encoding="latin1")
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: File not found. Please check the file path.")
except pd.errors.EmptyDataError:
    print("Error: The file is empty.")
except pd.errors.ParserError:
    print("Error: The file content could not be parsed.")
except PermissionError:
    print("Error: Permission denied when accessing the file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Data loaded successfully.


<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> 📌 Initial data inspection to understand the structure of the dataset and detect obvious issues.</p>

In [3]:
# Show DataFrame info to check the number of rows and columns, data types and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18919 entries, 0 to 18918
Columns: 1374 entries, DUID to VARPSU
dtypes: float64(1359), object(15)
memory usage: 198.3+ MB


In [4]:
# Show top five rows of the data
df.head()

,DUID,PID,DUPERSID,PANEL,DATAYEAR,FAMID31,FAMID42,FAMID53,FAMID23,FAMIDYR,...,RXWCP23,RXOSR23,RXPTR23,RXOTH23,PERWT23F,FAMWT23F,FAMWT23C,SAQWT23F,VARSTR,VARPSU
0,2790002.0,101.0,2790002101,27.0,2023.0,A,A,A,A,A,...,0.0,0.0,82.0,0.0,11664.426815,11158.817826,11158.817826,13221.315673,2019.0,1.0
1,2790002.0,102.0,2790002102,27.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,32212.113596,11158.817826,11158.817826,0.000000,2019.0,1.0
2,2790004.0,101.0,2790004101,27.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,21944.142826,28540.745942,28540.745942,29999.277476,2084.0,1.0
3,2790006.0,101.0,2790006101,27.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,10328.009530,10821.040689,10821.040689,11144.513916,2113.0,1.0
4,2790006.0,102.0,2790006102,27.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,17430.521357,10821.040689,10821.040689,0.000000,2113.0,1.0


<div style="background-color:#2c699d; color:white; padding:15px; border-radius:6px;">
    <h1 style="margin:0px">Data Preprocessing</h1>
</div> 

<div style="background-color:#fff6e4; padding:15px; border:3px solid #f5ecda; border-radius:6px;">
    <strong>Note</strong>: Kept column names in ALL CAPS in this project to ensure consistency with official <b><a href="../docs/references/h251doc.pdf">MEPS documentation</a></b>, <b><a href="../docs/references/h251cb.pdf">codebook</a></b>, and <b><a href="../docs/references/data_dictionary.md">data dictionary</a></b>.
</div>

<div style="background-color:#3d7ab3; color:white; padding:12px; border-radius:6px;">
    <h2 style="margin:0px">Handling Duplicates</h2>
</div>

<div style="background-color:#fff6e4; padding:15px; border:3px solid #f5ecda; border-radius:6px;">
    📌 Identify duplicates based on:
<ul>
    <li><strong>All columns</strong>: To detect exactly identical rows.</li>
    <li><strong>ID column only</strong>: To ensure that no two people share the same ID.</li>
    <li><strong>All columns except ID</strong>: To catch "hidden" duplicates where the same respondent may have been recorded twice under different IDs.</li>
</ul>
</div>

In [5]:
# Identify duplicates based on all columns
df.duplicated().value_counts()

False    18919
Name: count, dtype: int64

In [6]:
# Identify duplicates based on the ID column
df.duplicated(["DUPERSID"]).value_counts()

False    18919
Name: count, dtype: int64

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> ✅ No duplicates were found based on all columns or the ID column.</p>

In [7]:
# Identify duplicates based on all columns except ID columns
id_columns = ["DUPERSID", "DUID", "PID", "PANEL"]
duplicates_without_id = df.duplicated(subset=df.columns.drop(id_columns), keep=False)
duplicates_without_id.value_counts()

False    18913
True         6
Name: count, dtype: int64

In [8]:
# Show duplicates
df[duplicates_without_id]

,DUID,PID,DUPERSID,PANEL,DATAYEAR,FAMID31,FAMID42,FAMID53,FAMID23,FAMIDYR,...,RXWCP23,RXOSR23,RXPTR23,RXOTH23,PERWT23F,FAMWT23F,FAMWT23C,SAQWT23F,VARSTR,VARPSU
13345,2814572.0,108.0,2814572108,28.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,27691.899266,20267.048995,20267.048995,0.0,2081.0,2.0
13346,2814572.0,109.0,2814572109,28.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,27691.899266,20267.048995,20267.048995,0.0,2081.0,2.0
13635,2814836.0,104.0,2814836104,28.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,17440.683451,10104.630393,10104.630393,0.0,2018.0,6.0
13636,2814836.0,105.0,2814836105,28.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,17440.683451,10104.630393,10104.630393,0.0,2018.0,6.0
15314,2816427.0,104.0,2816427104,28.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,13323.385276,10928.475587,10928.475587,0.0,2070.0,2.0
15315,2816427.0,105.0,2816427105,28.0,2023.0,A,A,A,A,A,...,0.0,0.0,0.0,0.0,13323.385276,10928.475587,10928.475587,0.0,2070.0,2.0


<div style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px">
    💡 There are 3 pairs (or 6 rows) of duplicates that have identical values across all 1,300+ columns except for their IDs. Analysis shows these are young siblings (ages 1 and 5) living in the same household with identical parent-reported health data, identical sample weights, and identical costs. Thus, they appear to be valid respondents rather than "ghost records". However, they will be naturally excluded when filtering for the adult target population.
</div>

<div style="background-color:#3d7ab3; color:white; padding:12px; border-radius:6px;">
    <h2 style="margin:0px">Variable Selection</h2>
</div>

<div style="background-color:#fff6e4; padding:15px; border:3px solid #f5ecda; border-radius:6px;">
    📌 Filter the following 29 columns (out of 1,374):
    <ul style="margin-bottom:0px">
        <li><b>ID</b>: Unique identifier for each respondent (<code>DUPERSID</code>).</li>
        <li><b>Sample Weights</b>: Ensures population representativeness (<code>PERWT23F</code>).</li>
        <li><b>Candidate Features</b>: 26 variables selected for their consumer accessibility, beginning-of-year measurement, and predictive power.</li> 
        <li><b>Target Variable</b>: Total out-of-pocket health care costs (<code>TOTSLF23</code>).</li>
    </ul>
    <br>
    <b>Rationale</b>: For a detailed breakdown of the target variable selection and feature selection criteria, see the <b><a href="../docs/specs/technical_specifications.md">Technical Specifications</a></b> and <b><a href="../docs/research/candidate_features.md">Candidate Features Research</a></b>.
</div>

In [9]:
# List of columns to keep 
columns_to_keep = [
    # 1. ID
    "DUPERSID",
    
    # 2. Sample Weights
    "PERWT23F", 

    # 3 Candidate Features.
    # 3.1 Demographics
    "AGE23X", "SEX", "REGION23", "MARRY31X",
    
    # 3.2 Socioeconomic
    "POVCAT23", "FAMSZE23", "HIDEG", "EMPST31",
    
    # 3.3 Insurance & Access
    "INSCOV23", "HAVEUS42",
    
    # 3.4 Perceived Health & Lifestyle
    "RTHLTH31", "MNHLTH31", "ADSMOK42",
    
    # 3.5 Limitations & Symptoms
    "ADLHLP31", "IADLHP31", "WLKLIM31", "COGLIM31", "JTPAIN31_M18",
    
    # 3.6 Chronic Conditions
    "HIBPDX", "CHOLDX", "DIABDX_M18", "CHDDX", "STRKDX", "CANCERDX", "ARTHDX", "ASTHDX", 
    
    # 4. Healthcare Expenditure (Target)
    "TOTSLF23"
]

# Drop all other columns (keeping 29 out of 1,374)
df = df[columns_to_keep]

<div style="background-color:#3d7ab3; color:white; padding:12px; border-radius:6px;">
    <h2 style="margin:0px">Filtering Target Population</h2>
</div>

<div style="background-color:#fff6e4; padding:15px; border:3px solid #f5ecda; border-radius:6px;">
    📌 Filter rows to match the target population based on the following criteria:
    <ul style="margin-bottom:0px">
        <li><b>Positive person weight</b> (<code>PERWT23F > 0</code>): Drop respondents with a person weight of zero (i.e., 456 respondents). These individuals are considered "out-of-scope" for the full-year population (e.g., they joined the military, were institutionalized, or moved abroad).</li>
        <li><b>Adults</b> (<code>AGE23X >= 18</code>): Drop respondents under age 18 (i.e., 3796 respondents), as the medical cost planner app targets adults.</li>
    </ul>
    <br>
    <b>Note</b>: Keeps 14,768 out of 18,919 respondents.
</div>

In [10]:
# Filter DataFrame 
df = df[(df["PERWT23F"] > 0) & (df["AGE23X"] >= 18)].copy() 

<div style="background-color:#3d7ab3; color:white; padding:12px; border-radius:6px;">
    <h2 style="margin:0px">Handling Data Types</h2>
</div>

<div style="background-color:#fff6e4; padding:15px; border:3px solid #f5ecda; border-radius:6px;">
    📌 Identify and convert incorrect storage data types.
    <ul>
        <li><b>ID</b>: <code>DUPERSID</code> is an identifier, not a quantity. Converting them to <code>string</code> prevents unintended math.</li>
        <li><b>Sample Weights</b>: <code>PERWT23F</code> contains decimal precision critical for population-level estimates. Must remain <code>float</code>.</li>
        <li><b>Candidate Features</b>: The SAS loader stored all 26 features as floats by default. All features are either numerical or integer-encoded categorical variables. Converting them from <code>float</code> to <code>int</code> ensures proper representation and correctly handles missing codes (e.g., -1, -7).</li>
        <li><b>Target</b>: <code>TOTSLF23</code> is rounded to whole dollars in the MEPS Full Year data files (format 6.0). Can be safely stored as <code>int</code>.</li>
    </ul>
</div>

In [11]:
# Identify storage data types (defaulted to float/object by SAS loader)
df.dtypes

DUPERSID         object
PERWT23F        float64
AGE23X          float64
SEX             float64
REGION23        float64
MARRY31X        float64
POVCAT23        float64
FAMSZE23        float64
HIDEG           float64
EMPST31         float64
INSCOV23        float64
HAVEUS42        float64
RTHLTH31        float64
MNHLTH31        float64
ADSMOK42        float64
ADLHLP31        float64
IADLHP31        float64
WLKLIM31        float64
COGLIM31        float64
JTPAIN31_M18    float64
HIBPDX          float64
CHOLDX          float64
DIABDX_M18      float64
CHDDX           float64
STRKDX          float64
CANCERDX        float64
ARTHDX          float64
ASTHDX          float64
TOTSLF23        float64
dtype: object

In [12]:
# Convert ID to string
df["DUPERSID"] = df["DUPERSID"].astype(str)

# Convert feature and target columns from float to integer
int_cols = df.columns.drop(["DUPERSID", "PERWT23F"])
df[int_cols] = df[int_cols].astype(int)

# Verify the changes
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14768 entries, 0 to 18918
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DUPERSID      14768 non-null  object 
 1   PERWT23F      14768 non-null  float64
 2   AGE23X        14768 non-null  int64  
 3   SEX           14768 non-null  int64  
 4   REGION23      14768 non-null  int64  
 5   MARRY31X      14768 non-null  int64  
 6   POVCAT23      14768 non-null  int64  
 7   FAMSZE23      14768 non-null  int64  
 8   HIDEG         14768 non-null  int64  
 9   EMPST31       14768 non-null  int64  
 10  INSCOV23      14768 non-null  int64  
 11  HAVEUS42      14768 non-null  int64  
 12  RTHLTH31      14768 non-null  int64  
 13  MNHLTH31      14768 non-null  int64  
 14  ADSMOK42      14768 non-null  int64  
 15  ADLHLP31      14768 non-null  int64  
 16  IADLHP31      14768 non-null  int64  
 17  WLKLIM31      14768 non-null  int64  
 18  COGLIM31      14768 non-null  i

<div style="background-color:#2c699d; color:white; padding:15px; border-radius:6px;">
    <h2 style="margin:0px">Standardizing Missing Values</h1>
</div> 

<div style="background-color:#e8f4fd; padding:15px; border:3px solid #d0e7fa; border-radius:6px;">
    ℹ️ MEPS Missing Value Codes:
    <ul style="margin-bottom:0px">
        <li><b>-1 INAPPLICABLE</b>: Variable does not apply (structural skip).</li>
        <li><b>-7 REFUSED</b>: Person refused to answer.</li>
        <li><b>-8 DON'T KNOW</b>: Person did not know the answer.</li>
        <li><b>-9 NOT ASCERTAINED</b>: Administrative or technical error in collection.</li>
        <li><b>-15 CANNOT BE COMPUTED</b>: Incomplete data for a constructed variable.</li>
    </ul>
</div>

In [13]:
# Identify standard missing values (np.nan, pd.NA, None, pd.NaT)
df.isnull().sum()

DUPERSID        0
PERWT23F        0
AGE23X          0
SEX             0
REGION23        0
MARRY31X        0
POVCAT23        0
FAMSZE23        0
HIDEG           0
EMPST31         0
INSCOV23        0
HAVEUS42        0
RTHLTH31        0
MNHLTH31        0
ADSMOK42        0
ADLHLP31        0
IADLHP31        0
WLKLIM31        0
COGLIM31        0
JTPAIN31_M18    0
HIBPDX          0
CHOLDX          0
DIABDX_M18      0
CHDDX           0
STRKDX          0
CANCERDX        0
ARTHDX          0
ASTHDX          0
TOTSLF23        0
dtype: int64

<div style="background-color:#fff6e4; padding:15px; border:3px solid #f5ecda; border-radius:6px;">
    📌 Handle MEPS-specific missing and skip patterns.
</div>

<div style="background-color:#fff6e4; padding:15px; border:3px solid #f5ecda; border-radius:6px;">
    Skip patterns: Address high "Inapplicable" (-1) rates by recovering "ground truth" values defined by MEPS structural skip patterns.  
    <ul>
        <li><b>Smoking (ADSMOK42)</b>: This question is only asked if the respondent answered "Yes" to the gateway question: <i>"Have you smoked at least 100 cigarettes in your life?"</i>. Respondents who answered "No" are coded as -1. For the Medical Cost Planner app, these "Never Smokers" are functionally <b>"No" (2)</b>.</li>
        <li><b>Joint Pain (JTPAIN31_M18)</b>: This question is skipped for adults who already reported an <b>Arthritis Diagnosis (ARTHDX)</b> earlier in the interview. Since arthritis inherently involves joint symptoms, these -1 values are logically recovered as <b>"Yes" (1)</b>.</li>
    </ul>
</div>

In [14]:
# Identify MEPS-specific missing values 
missing_codes = [-1, -7, -8, -9, -15]
missing_frequency_df = pd.DataFrame({code: (df == code).sum() for code in missing_codes})
missing_frequency_df["TOTAL"] = missing_frequency_df.sum(axis=1)
missing_frequency_df["PERCENTAGE"] = (missing_frequency_df["TOTAL"] / len(df) * 100).round(2)
missing_frequency_df.sort_values("TOTAL", ascending=False) 

,-1,-7,-8,-9,-15,TOTAL,PERCENTAGE
ADSMOK42,4992,0,0,0,386,5378,36.42
JTPAIN31_M18,4059,23,69,0,0,4151,28.11
HAVEUS42,8,207,338,0,0,553,3.74
EMPST31,7,154,45,0,73,279,1.89
COGLIM31,154,0,26,0,0,180,1.22
HIDEG,0,29,105,0,0,134,0.91
CHOLDX,79,2,23,0,0,104,0.70
HIBPDX,79,1,17,0,0,97,0.66
CANCERDX,79,1,9,0,1,90,0.61
ARTHDX,79,1,10,0,0,90,0.61


In [ ]:
# PERFORM LOGIC RECOVERY
# 1. Recover Smoker Status: Map -1 (Never smokers) to 2 (No)
df.loc[df["ADSMOK42"] == -1, "ADSMOK42"] = 2

# 2. Recover Joint Pain: Map -1 to 1 (Yes) only if they have an Arthritis Diagnosis
df.loc[(df["JTPAIN31_M18"] == -1) & (df["ARTHDX"] == 1), "JTPAIN31_M18"] = 1

In [ ]:
# Convert all remaining MEPS missing codes to np.nan
df = df.replace(missing_codes, np.nan)